In [1]:
from bs4 import BeautifulSoup, Comment
from urllib.request import urlopen
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive # Import only if you are using Google Colab

In [2]:
seasons = list(range(1980, 2022))

In [3]:
basic_stats_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')][1:]

    rows = soup.findAll('tr', class_ = lambda x: x != 'thead')[1:]
    players_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

    stats = pd.DataFrame(players_stats, columns = headers)
    stats['Season'] = season

    basic_stats_per_season.append(stats)

basic_stats = pd.concat(basic_stats_per_season)

In [4]:
basic_stats.tail()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Season
700,Delon Wright,PG,28,SAC,27,8,25.8,3.9,8.3,.462,1.2,3.1,.398,2.6,5.3,.500,.536,1.1,1.3,.833,1.0,2.9,3.9,3.6,1.6,0.4,1.3,1.1,10.0,2021
701,Thaddeus Young,PF,32,CHI,68,23,24.3,5.4,9.7,.559,0.2,0.7,.267,5.3,9.1,.580,.568,1.0,1.7,.628,2.5,3.8,6.2,4.3,1.1,0.6,2.0,2.2,12.1,2021
702,Trae Young,PG,22,ATL,63,63,33.7,7.7,17.7,.438,2.2,6.3,.343,5.6,11.3,.491,.499,7.7,8.7,.886,0.6,3.3,3.9,9.4,0.8,0.2,4.1,1.8,25.3,2021
703,Cody Zeller,C,28,CHO,48,21,20.9,3.8,6.8,.559,0.1,0.6,.143,3.7,6.2,.598,.565,1.8,2.5,.714,2.5,4.4,6.8,1.8,0.6,0.4,1.1,2.5,9.4,2021
704,Ivica Zubac,C,23,LAC,72,33,22.3,3.6,5.5,.652,0.0,0.1,.250,3.6,5.4,.656,.654,1.9,2.4,.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,2021


In [5]:
advanced_stats_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')][1:]

    rows = soup.findAll('tr', class_ = lambda x: x != 'thead')[1:]
    players_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

    stats = pd.DataFrame(players_stats, columns = headers)
    stats['Season'] = season

    advanced_stats_per_season.append(stats)

advanced_stats = pd.concat(advanced_stats_per_season)

In [6]:
advanced_stats.tail()

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,,OWS,DWS,WS,WS/48,,OBPM,DBPM,BPM,VORP,Season
700,Delon Wright,PG,28,SAC,27,696,15.9,.563,.369,.160,4.4,12.5,8.4,18.9,3.0,1.3,12.7,17.1,,0.8,0.5,1.3,.092,,0.5,0.6,1.2,0.6,2021
701,Thaddeus Young,PF,32,CHI,68,1652,20.3,.578,.068,.171,11.4,16.9,14.2,27.1,2.2,2.1,16.1,22.3,,2.8,2.2,5.1,.147,,1.9,1.4,3.3,2.2,2021
702,Trae Young,PG,22,ATL,63,2125,23.0,.589,.357,.491,2.0,10.5,6.3,45.5,1.2,0.5,16.2,33.0,,5.9,1.3,7.2,.163,,5.3,-1.7,3.7,3.0,2021
703,Cody Zeller,C,28,CHO,48,1005,18.2,.599,.086,.367,12.6,22.9,17.7,13.2,1.3,1.7,11.9,18.3,,2.1,1.1,3.3,.156,,-0.2,-0.2,-0.5,0.4,2021
704,Ivica Zubac,C,23,LAC,72,1609,19.1,.693,.010,.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,,4.8,2.1,6.9,.206,,0.6,0.4,1.0,1.2,2021


In [7]:
mvp_award_voting_per_season = []
for season in seasons[:-1]:
    url = 'https://www.basketball-reference.com/awards/awards_{}.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('tr', limit = 2)[1].findAll('th')][1:]

    table = soup.find(lambda tag: tag.has_attr('id') and tag['id'] == 'mvp')
    rows = table.findAll('tr', class_ = lambda x: x != 'thead')[1:]
    players_stats = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

    stats = pd.DataFrame(players_stats, columns = headers)
    stats['Season'] = season

    mvp_award_voting_per_season.append(stats)

mvp_award_voting = pd.concat(mvp_award_voting_per_season)

In [8]:
mvp_award_voting.tail()

,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Season
8,Damian Lillard,29,POR,0.0,23.0,1010,0.023,66,37.5,30.0,4.3,8.0,1.1,0.3,.463,.401,.888,11.6,.225,2020
9,Nikola Jokić,24,DEN,0.0,18.0,1010,0.018,73,32.0,19.9,9.7,7.0,1.2,0.6,.528,.314,.817,9.8,.202,2020
10,Pascal Siakam,25,TOR,0.0,17.0,1010,0.017,60,35.2,22.9,7.3,3.5,1.0,0.9,.453,.359,.792,5.4,.123,2020
11,Jimmy Butler,30,MIA,0.0,9.0,1010,0.009,58,33.8,19.9,6.7,6.0,1.8,0.6,.455,.244,.834,9.0,.221,2020
12,Jayson Tatum,21,BOS,0.0,1.0,1010,0.001,66,34.3,23.4,7.0,3.0,1.4,0.9,.450,.403,.812,6.9,.146,2020


In [9]:
standings_per_season = []
for season in seasons:
    url = 'https://www.basketball-reference.com/leagues/NBA_{}_standings.html'.format(season)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    commented_html = []
    for comments in soup.findAll(text = lambda text:isinstance(text, Comment)):
        commented_html.append(comments.extract())
    commented_soup = BeautifulSoup(commented_html[28])

    headers = [th.getText() for th in commented_soup.findAll('tr', limit = 2)[1].findAll('th')][:3]

    rows = commented_soup.findAll('tr')[2:]
    teams_stats = [[td.getText() for td in rows[i].findAll(lambda tag: tag.has_attr('data-stat') and tag['data-stat'] == 'ranker' or 'team_name' or 'Overall')][:4] for i in range(len(rows))]
    for team_stat in teams_stats:
        team_stat.pop(2)

    stats = pd.DataFrame(teams_stats, columns = headers)
    stats['Season'] = season

    standings_per_season.append(stats)

standings_stats = pd.concat(standings_per_season)

In [10]:
standings_stats.tail()

,Rk,Team,Overall,Season
25,26,Cleveland Cavaliers,22-50,2021
26,27,Oklahoma City Thunder,22-50,2021
27,28,Orlando Magic,21-51,2021
28,29,Detroit Pistons,20-52,2021
29,30,Houston Rockets,17-55,2021


In [11]:
teams = {'Atlanta Hawks': 'ATL',
         'Boston Celtics': 'BOS',
         'Brooklyn Nets': 'BRK',
         'Charlotte Bobcats': 'CHA',
         'Charlotte Hornets': 'CHH/CHO',
         'Chicago Bulls': 'CHI',
         'Cleveland Cavaliers': 'CLE',
         'Dallas Mavericks': 'DAL',
         'Denver Nuggets': 'DEN',
         'Detroit Pistons': 'DET',
         'Golden State Warriors': 'GSW',
         'Houston Rockets': 'HOU',
         'Indiana Pacers': 'IND',
         'Kansas City Kings': 'KCK',
         'Los Angeles Clippers': 'LAC',
         'Los Angeles Lakers': 'LAL',
         'Memphis Grizzlies': 'MEM',
         'Miami Heat': 'MIA',
         'Milwaukee Bucks': 'MIL',
         'Minnesota Timberwolves': 'MIN',
         'New Jersey Nets': 'NJN',
         'New Orleans Hornets': 'NOH',
         'New Orleans/Oklahoma City Hornets': 'NOK',
         'New Orleans Pelicans': 'NOP',
         'New York Knicks': 'NYK',
         'Oklahoma City Thunder': 'OKC',
         'Orlando Magic': 'ORL',
         'Philadelphia 76ers': 'PHI',
         'Phoenix Suns': 'PHO',
         'Portland Trail Blazers': 'POR',
         'Sacramento Kings': 'SAC',
         'San Antonio Spurs': 'SAS',
         'San Diego Clippers': 'SDC',
         'Seattle SuperSonics': 'SEA',
         'Toronto Raptors': 'TOR',
         'Utah Jazz': 'UTA',
         'Vancouver Grizzlies': 'VAN',
         'Washington Wizards': 'WAS',
         'Washington Bullets': 'WSB'}

In [12]:
def transform_team_column(x: str) -> str:
    return teams[x]

standings_stats.Team = standings_stats.Team.apply(transform_team_column)

In [13]:
maskChh = (standings_stats.Team == 'CHH/CHO') & (standings_stats.Season <= 2002)
standings_stats.Team[maskChh] = 'CHH'

maskCho = (standings_stats.Team == 'CHH/CHO') & (standings_stats.Season >= 2015)
standings_stats.Team[maskCho] = 'CHO'

In [14]:
def transform_player_column(x: str) -> str:
    return x.replace('*', '')

basic_stats.Player = basic_stats.Player.apply(transform_player_column)
advanced_stats.Player = advanced_stats.Player.apply(transform_player_column)

In [15]:
basic_stats['PlayerTmSeason'] = basic_stats.Player.map(str) + basic_stats.Tm.map(str) + basic_stats.Season.map(str)
basic_stats['TmSeason'] = basic_stats.Tm.map(str) + basic_stats.Season.map(str)

advanced_stats['PlayerTmSeason'] = advanced_stats.Player.map(str) + advanced_stats.Tm.map(str) + advanced_stats.Season.map(str)
mvp_award_voting['PlayerTmSeason'] = mvp_award_voting.Player.map(str) + mvp_award_voting.Tm.map(str) + mvp_award_voting.Season.map(str)
standings_stats['TmSeason'] = standings_stats.Team.map(str) + standings_stats.Season.map(str)

In [16]:
df = basic_stats.merge(advanced_stats, on = 'PlayerTmSeason', how = 'left')
df = df.merge(mvp_award_voting, on = 'PlayerTmSeason', how = 'left')
df = df.merge(standings_stats, on = 'TmSeason', how = 'left')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21657 entries, 0 to 21656
Data columns (total 85 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player_x        21657 non-null  object 
 1   Pos_x           21657 non-null  object 
 2   Age_x           21657 non-null  object 
 3   Tm_x            21657 non-null  object 
 4   G_x             21657 non-null  object 
 5   GS              21657 non-null  object 
 6   MP_x            21657 non-null  object 
 7   FG              21657 non-null  object 
 8   FGA             21657 non-null  object 
 9   FG%_x           21657 non-null  object 
 10  3P              21657 non-null  object 
 11  3PA             21657 non-null  object 
 12  3P%_x           21657 non-null  object 
 13  2P              21657 non-null  object 
 14  2PA             21657 non-null  object 
 15  2P%             21657 non-null  object 
 16  eFG%            21657 non-null  object 
 17  FT              21657 non-null 

In [18]:
df = df.loc[:, ~df.columns.duplicated()]

In [19]:
df.drop(columns = ['\xa0', 'PlayerTmSeason', 'TmSeason', 'Player_y', 'Pos_y', 'Age_y', 'Tm_y', 'G_y', 'MP_y', 'Season_y', 'Player', 'Age', 'Tm',
                   'G', 'MP', 'PTS_y', 'TRB_y', 'AST_y', 'STL_y', 'BLK_y', 'FG%_y', '3P%_y', 'FT%_y', 'WS_y', 'WS/48_y', 'Team'],
    inplace = True)

In [20]:
df.rename(columns = {'Player_x': 'Player', 'Pos_x': 'Pos', 'Age_x': 'Age', 'Tm_x': 'Tm', 'G_x': 'G', 'MP_x': 'MP',
                     'FG%_x': 'FG%', '3P%_x': '3P%', 'FT%_x': 'FT%', 'TRB_x': 'TRB', 'AST_x': 'AST', 'STL_x': 'STL',
                     'BLK_x': 'BLK', 'PTS_x': 'PTS', 'Season_x': 'Season', 'WS_x': 'WS', 'WS/48_x': 'WS/48'},
    inplace = True)

In [21]:
df.fillna({'G': 0, 'GS': 0, 'MP': 0, 'FG': 0, 'FGA': 0, 'FG%': 0, '3P': 0, '3PA': 0, '3P%': 0, '2P': 0,
           '2PA': 0, '2P%': 0, 'eFG%': 0, 'FT': 0, 'FTA': 0, 'FT%': 0, 'ORB': 0, 'DRB': 0, 'TRB': 0, 'AST': 0,
           'STL': 0, 'BLK': 0, 'TOV': 0, 'PF': 0, 'PTS': 0, 'PER': 0, 'TS%': 0, '3PAr': 0, 'FTr': 0, 'ORB%': 0,
           'DRB%': 0, 'TRB%': 0, 'AST%': 0, 'STL%': 0, 'BLK%': 0, 'TOV%': 0, 'USG%': 0, 'OWS': 0, 'DWS': 0, 'WS': 0,
           'WS/48': 0, 'OBPM': 0, 'DBPM': 0, 'BPM': 0, 'VORP': 0, 'First': 0, 'Pts Won': 0, 'Pts Max': 0, 'Share': 0},
    inplace = True)

In [22]:
df = df[df.Tm != 'TOT']

In [23]:
int_columns = ['Age', 'G', 'GS', 'First', 'Pts Won', 'Pts Max', 'Rk']
percentage_float_columns = ['FG%', '3P%', '2P%', 'eFG%', 'FT%', 'TS%', '3PAr', 'FTr', 'WS/48', 'Share']
float_columns = ['MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
                 'PTS', 'PER', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP']

for column in int_columns:
    if column == 'GS':
        df[column] = df[column].astype(str).apply(lambda x: '0' if x == '' else x)

    if column == 'First' or 'Pts Won':
        df[column] = df[column].astype(str).apply(lambda x: x[:-2] if '.' in x else x)

    df[column] = df[column].astype(int)

for column in percentage_float_columns:
    if column == 'Share':
        df[column] = df[column].astype(str).apply(lambda x: '0.0' if x == '0' else x).astype(float)
    else:
        df[column] = df[column].apply(lambda x: x.zfill(1)).astype(float)

for column in float_columns:
    if column == 'MP' or 'PER' or 'USG%':
        df[column] = df[column].astype(str).apply(lambda x: '0.0' if x == '' else x)

    df[column] = df[column].astype(float)

In [24]:
df_train = df[df.Season <= 2020]

df_production = df[df.Season == 2021]

In [25]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19111 entries, 0 to 20951
Data columns (total 56 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   19111 non-null  object 
 1   Pos      19111 non-null  object 
 2   Age      19111 non-null  int64  
 3   Tm       19111 non-null  object 
 4   G        19111 non-null  int64  
 5   GS       19111 non-null  int64  
 6   MP       19111 non-null  float64
 7   FG       19111 non-null  float64
 8   FGA      19111 non-null  float64
 9   FG%      19111 non-null  float64
 10  3P       19111 non-null  float64
 11  3PA      19111 non-null  float64
 12  3P%      19111 non-null  float64
 13  2P       19111 non-null  float64
 14  2PA      19111 non-null  float64
 15  2P%      19111 non-null  float64
 16  eFG%     19111 non-null  float64
 17  FT       19111 non-null  float64
 18  FTA      19111 non-null  float64
 19  FT%      19111 non-null  float64
 20  ORB      19111 non-null  float64
 21  DRB      191

In [26]:
df_production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626 entries, 20952 to 21656
Data columns (total 56 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Player   626 non-null    object 
 1   Pos      626 non-null    object 
 2   Age      626 non-null    int64  
 3   Tm       626 non-null    object 
 4   G        626 non-null    int64  
 5   GS       626 non-null    int64  
 6   MP       626 non-null    float64
 7   FG       626 non-null    float64
 8   FGA      626 non-null    float64
 9   FG%      626 non-null    float64
 10  3P       626 non-null    float64
 11  3PA      626 non-null    float64
 12  3P%      626 non-null    float64
 13  2P       626 non-null    float64
 14  2PA      626 non-null    float64
 15  2P%      626 non-null    float64
 16  eFG%     626 non-null    float64
 17  FT       626 non-null    float64
 18  FTA      626 non-null    float64
 19  FT%      626 non-null    float64
 20  ORB      626 non-null    float64
 21  DRB      6

In [27]:
drive.mount('drive')

df_train.to_csv('df_1980_2020.csv', index = False)
!cp df_1980_2020.csv 'drive/My Drive/'

df_production.to_csv('df_2021.csv', index = False)
!cp df_2021.csv 'drive/My Drive/'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
